## Языковая Модель GPT-2

Смотри лекцию по теме по ссылке: https://docs.google.com/presentation/d/1AcKnTMw0L6PZX9LK2JJkVIx5rgOSU2s6lL9DSfE6yNA/edit?usp=sharing

- Что такое gpt-2
  - архитектура
  - входы выходы

  <br></br>
<img src="https://drive.google.com/uc?export=view&id=13ya5tFwrHpD7x9BBSjx3G0ojiMwvmR3L" width=500>  
<br></br>

GPT-2 - это трансформер, который был обучен предсказывать следующий токен на огромном датасете текстов. В основе данной архитектуры лежит несколько transformer-блоков, каждый из которых передаёт свой выход на вход следующему блоку. 

В GPT используется необычный токенайзер, который называется BPE-tokenizer. Если слово есть в словаре токенайзера то оно превращается в один индекс, если его нет в словаре, то слово разбивается на составные части и снова ищется в словаре. Это происходит до тех пор, пока мы не найдём все составные части слова в словаре.

Словарь GPT изначально заполняется всеми одиночными символами, а после дополняется наиболее частыми последовательностями символов.


### Задача, на которой обучался GPT

Как уже было сказано, GPT обучался на задаче предсказания следующих токенов по предыдущим, эта задача называется Language Modelling. 

<br></br>
<img src="https://drive.google.com/uc?export=view&id=1_oG5CLmFmCTlWO8te1KoAf8BTpvSxHX7" width=500>  
<br></br>

А именно, мы максимизируем вероятность правильного токена при условии предыдущих токенов.




Мы будем работать с библиотекой `transformers` от HuggingFace и использовать её `torch` backend. Для загрузки данных возьмём библиотеку `datasets` от тех же разработчиков.  

Устанвим их с помощью `pip`:

In [ ]:
!pip install transformers datasets

Импортируем всё что нам понадобится заранее, однако, пока нам нужны только `AutoTokenizer` и `AutoModelForCausalLM`

In [ ]:
import json

from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, default_data_collator
import transformers
import pandas as pd
from tqdm import tqdm

Для наших экспериментов возьмём небольшую русскую модель GPT-3 от сбера: https://huggingface.co/sberbank-ai/rugpt3small_based_on_gpt2

Можно так же взять [среднюю](https://huggingface.co/sberbank-ai/rugpt3medium_based_on_gpt2) или [большую](https://huggingface.co/sberbank-ai/rugpt3large_based_on_gpt2) модель, однако, они будут заметно дольше считаться. 

Можно посмотреть и другие модели, например, GPT-2 от них же: https://huggingface.co/sberbank-ai.

Для работы нам понадобится сама модель и токенизатор. 

In [ ]:
model = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

GPT - огромные модели, размеры большого варианта модели приближаются к миллиарду параметров.

In [ ]:
total_params = 0
for p in model.parameters():
    total_params += torch.prod(torch.tensor(p.shape))

print(f'Количество параметров сети: {total_params}')

Чтобы определить какой именно класс был использован для построения модели, что полезно для поиска документации, можно использовать встроенную функцию `type`

In [ ]:
type(model), type(tokenizer)

По этим классам легко найти [документацию](https://huggingface.co/docs/transformers/model_doc/gpt2).

Попробуем разбить на токены какой-нибудь текст

In [ ]:
text = 'Сосиска в тексте'
tokenizer.tokenize(text)

К сожалению, из-за того, что все символы, исходно представленные в кодировке `utf-8`, разбиты на байты BPE токенизатором, результ малочитаемый. Однако, что-то полезное уже можно увидеть из разбиения на токены, а именно, что слова было три, а токенов получилось 5. Видимо, слово "Сосиска" с большой буквы не так уж часто встречается в текстах, поэтому для него нет отдельного токена. Для частых слов, как правило, в словаре уже есть отдельный токен.

In [ ]:
tokenizer.tokenize("клаустрофобия")

Для некоторых моделей результат токенизации читаемый, однако, можно попробовать их превратить в читаемые методом `convert_tokens_to_string`

In [ ]:
tokenizer.convert_tokens_to_string(['ÐºÐ»Ð°', 'ÑĥÑģÑĤÑĢÐ¾'])

**Задача 0**

Найдите пару слов которые токенизируются в один токен и одно **существующее** слово, которое разбивается на наибольшее количество токенов

In [ ]:
tokenizer.tokenize("это")

Токенизация - это лишь часть подготовки текста для подачи в модель, за ней следует перевод токенов в индексы. Перевод в индексы осуществляется по словарю, который хранится в токенайзере. Словарь вида `{'a': 0, 'ab': 1, ...}`. По этим индексам модель возьмёт соответствующие вектора токенов.

In [ ]:
text = 'Сосиска в тексте'
tokens = tokenizer.tokenize(text)
tokenizer.convert_tokens_to_ids(tokens)

Практически в таком виде модель принимает входные данные, остаётся только конвертировать индексы в тензор и добавить `batch` размерность. Всё это вместе можно сделать одним уже готовым методом:

In [ ]:
batch = tokenizer(text, return_tensors='pt')
print(batch)

Вместе с индексами токенов токенизатор вернул и маску для attention-а, в случае, когда текст один, она не несёт информативной нагрузки, однако, когда в batch-е встречаются тексты разной длины, она позволяет не смотреть на паддинги (дополнения неинформативным символом текстов до одной длины в токенах).

<br></br>

Тензоры, полученные на выходе токенизатора, многомерные. Часто удобно посмотреть на их размерности чтобы понять что к чему. 

**Задача 1**

Выведем эти размерности:

In [ ]:
print(batch['input_ids'].shape)

# ВАШ КОД, который выводит размерность маски

Перейдём к модели. Для начала, перенесём её на GPU, иначе считаться на CPU будет очень долго.

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu:0')

model.to(device)

Перенесём полученный входной батч на GPU и подадим его в модель.

In [ ]:
batch_gpu = batch.to(device)
outs = model(**batch_gpu)
outs

В outs лежит словарь с тензороми, посмотрим ключи словаря.

In [ ]:
outs.keys()

Нас интересует тензор `logits`, это логарифм вероятностей следующего токена.

**Задача 2**

Посмотрим на размерности этого тезора.

In [ ]:
print(outs['logits'] # ВАШ КОД ЗДЕСЬ)


Обратите внимание, что для данной модели сначала идет размерность `batch_size`, а потом уже `max_seq_len`. Расскажите соседу, как взять вероятность токена который мы будем генерировать следующим.

**Задача 2** * (можно пропустить)

Вариант еще сложнее:

- по лог-вероятностям вычислить вероятности (здесь нужен `torch.softmax(###, dim=seq_dim)`, где `seq_dim` - номер размерности, имеющей размер словаря).
- выбрать последний токен в последовательности 
- засемплировать следующий токен с помощью примерно такого кода
```python
In [16]: torch.distributions.Categorical(torch.tensor([0.1, 0.2, 0.7])).sample()
Out[16]: tensor(2)
```
вероятности подставить свои.



Теперь попробуем сгенерировать что-нибудь, подав на вход наш текст `Сосиска в тексте`

In [ ]:
max_length = 100
do_sample = True
top_k = 50
top_p = 0.95

outputs = model.generate(**batch_gpu, 
                         return_dict_in_generate=True, # Это важный параметр!
                         output_scores=True, # Это важный параметр!
                         top_k=top_k,
                         top_p=top_p,
                         do_sample=do_sample, 
                         max_length=max_length)

In [ ]:
tokenizer.batch_decode(outputs.sequences)

**Задача 3**

- задайте свой текст
- токенизируйте его
- поместите батч на GPU
- сгенерируйте продолжение моделью
- длина сгенерированного текста должна быть не меньше 32

### Дообучение модели

Для дообучения модели нам нужны тексты. Для примера создадим маленький датасет.

In [ ]:
with open('dataset.txt', 'w') as fp:
    fp.write('Это первый текст да\nэто второй текст')

with open('dataset_test.txt', 'w') as fp:
    fp.write('Это вовсе не текст\nэто может быть второй текст')

Тут всего две строчки, будем считать что это два семпла (хотя могли бы взять обе строчки как один семпл). Каждый текст положим в отдельный словарь по ключу `input` и спарсим его в `json`-строчку.

In [ ]:
with open('dataset.json', 'w') as fp:
    with open('dataset.txt') as fpt:
        texts = fpt.read().split('\n')
    for text in texts:
        fp.write(json.dumps({'input': text})+ '\n')

with open('dataset_test.json', 'w') as fp:
    with open('dataset_test.txt') as fpt:
        texts = fpt.read().split('\n')
    for text in texts:
        fp.write(json.dumps({'input': text}) + '\n')

Создадим датасет из строк из получившегося `json`-a.

In [ ]:
raw_dataset = load_dataset('json', data_files={'train': 'dataset.json', 'test': 'dataset_test.json'})

Сделаем токенизацию датасета и добавим целевые переменные `labels`, они будут идентичны самими входам (библиотека `transformers` сама сдвинет последовательность для предсказания).

In [ ]:
def tokenize_function(example):
    return tokenizer(example['input'], return_tensors='pt')


def add_labels(example):
    example['labels'] = example['input_ids']
    return example


columns_to_remove = raw_dataset["train"].column_names
ds = raw_dataset.map(tokenize_function, remove_columns=columns_to_remove, load_from_cache_file=False).map(add_labels, load_from_cache_file=False)
ds['train'][0]

Посмотрим на размеры полученных входных индексов токенов

In [ ]:
ds['train'][0]['input_ids']

**Задача 4**

Выведите количество семплов в датасете

Сделаем `DataLoader`, который будет генерировать батчи. Для этого сначала напишем collate функцию, которая дополняет до нужной длины нулями:

In [ ]:
tokenizer.pad_token_id = 0
tokenizer.pad_token = '<pad>'
tokenizer.pad_token = tokenizer.get_vocab()


dl = DataLoader(ds['train'], batch_size=1, collate_fn=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))
dl_test = DataLoader(ds['test'], batch_size=1, collate_fn=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

Посмотрим что генерирует `Dataloader`

In [ ]:
next(iter(dl_test))

**Задача 4** *

Настройте `DataCollatorForLanguageModeling` для батча > 1 и текстов разной длины.

Давайте проверим, что выдает текущий gpt для входа 'Это'. Мы ожидаем, что до обучения gpt будет продолжать как-то, а после обучения как 'Это наш первый вариант':

In [ ]:
text = 'Это'

max_length = 100
do_sample = True
top_k = 50
top_p = 0.95

outputs = model.generate(**tokenizer(text, return_tensors='pt').to(device), 
                         return_dict_in_generate=True, # Это важный параметр!
                         output_scores=True, # Это важный параметр!
                         top_k=top_k,
                         top_p=top_p,
                         do_sample=do_sample, 
                         max_length=max_length)

In [ ]:
tokenizer.batch_decode(out.sequences)

Теперь мы можем готовить батчи, осталось подготовить оптимизитор и запустить обучение.

In [ ]:
lr = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr)

In [ ]:
epochs = 32

for epoch in tqdm(range(epochs)):
    total_loss = 0
    for batch in dl:
        optimizer.zero_grad()
        output = model(**batch.to(device))
        output.loss.backward()
        optimizer.step()
        total_loss += output.loss.item()

    # Оцениваем loss на тестовой выборке

    # Экономим пямять чтобы не хранить информацию для градиентов
    with torch.no_grad():
        total_loss_test = 0
        for batch in dl_test:
            output = model(**batch.to(device))
            total_loss_test += output.loss.item()
  
    print(f'Mean loss on epoch {epoch}: train {total_loss / len(dl):.3f} test {total_loss_test / len(dl_test):.3f}')

In [ ]:
text = 'Это'

max_length = 100
do_sample = True
top_k = 50
top_p = 0.95

outputs = model.generate(**tokenizer(text, return_tensors='pt').to(device), 
                         return_dict_in_generate=True, # Это важный параметр!
                         output_scores=True, # Это важный параметр!
                         top_k=top_k,
                         top_p=top_p,
                         do_sample=do_sample, 
                         max_length=max_length)

In [ ]:
tokenizer.batch_decode(out.sequences)

**Задача 5**

Найдите свой набор текстов и замените игрушечную пару предложений на него в коде 
```python
import json

with open('dataset.txt', 'w') as fp:
  fp.write('Это первый текст да\nэто второй текст')

with open('dataset.json', 'w') as fp:
  with open('dataset.txt') as fpt:
    texts = fpt.read().split('\n')
  for text in texts:
    fp.write(json.dumps({'input': text}))

```


